In [5]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, roc_curve
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor  
import warnings
import pickle
import os
import json

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
cols = ['ts', 'uid', 'id.orig_h', 'id.orig_p',
        'id.resp_h', 'id.resp_p', 'proto', 'service',
        'duration',  'orig_bytes', 'resp_bytes',
        'conn_state', 'local_orig', 'local_resp',
        'missed_bytes',  'history', 'orig_pkts',
        'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes',
        'tunnel_parents', 'label']

In [19]:
out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/cupid/Zeek/timeout60'
df = pd.read_csv(out_dir+"/CUPID_zeek_60.csv")
df.head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,Attack,id
0,1.556010e+09,C0APT221ALnx5VEm09,192.168.1.37,61532,192.168.1.13,80,tcp,http,0.313488,126,518,SF,T,T,0,ShADTadtFf,10,676,10,1460,-,Benign,NaN
1,1.556010e+09,C5fvroXMbm1PEX0A1,192.168.1.7,49487,192.168.1.5,135,tcp,-,0.000160,0,0,SF,T,T,0,FafA,4,160,4,160,-,Benign,NaN
2,1.556010e+09,C6WYBR2L7KPjPydTG8,192.168.1.38,57843,192.168.1.7,445,tcp,smb,0.001248,144,144,OTH,T,T,0,DTadtAR,8,608,6,528,-,Benign,NaN
3,1.556010e+09,CWT3g72MjovVr3nLc9,192.168.1.34,57856,192.168.1.7,88,tcp,krb_tcp,0.002567,1325,1340,RSTR,T,T,0,ShADTdtFar,8,2994,8,3024,-,Benign,NaN
4,1.556010e+09,CzdUxA1V9PVXc6lH7i,192.168.1.34,57857,192.168.1.7,88,tcp,-,0.001394,2337,115,RSTR,T,T,2338,ShaGADTdtFr,10,5098,10,654,-,Benign,NaN


In [16]:
df['duration'].unique()

array(['0.313488', '0.000160', '0.001248', ..., '3301.412548',
       '440.829365', '3300.928775'], dtype=object)

In [20]:
df.columns

Index(['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p',
       'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes',
       'conn_state', 'local_orig', 'local_resp', 'missed_bytes', 'history',
       'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes',
       'tunnel_parents', 'Attack', 'id'],
      dtype='object')

In [2]:
def save_scores(timeout, meanScores, stdScore):
    results = {
        'Timeout': timeout,
        'Mean of all scores': meanScores,
        'Std of all Scores': stdScores

    }

    with open(f'../Checkpoints/ET/ET_cupid_zeek_{timeout}.json', 'w') as f:
        json.dump(results, f, indent=4)

In [3]:
timeouts = ['default', 0.5, 1, 2, 3, 4, 5, 6, 10, 30, 60]

# Training

In [6]:
best_f1 = 0
worst_f1 = 1
best_mean, worst_mean, best_std, worst_std = None, None, None, None

save=True

for timeout in timeouts:
    print("Processing timeout : ", timeout)
    if timeout =='default':
        out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/cupid/Zeek/{timeout}/CUPID_zeek_{timeout}.csv'
    else:
        out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/cupid/Zeek/timeout{timeout}/CUPID_zeek_{timeout}.csv'

    df = pd.read_csv(out_dir)
    
    df = df.drop(columns=['id', 'uid', 'id.orig_h', 'id.resp_h', 'tunnel_parents']) # tunnel_parents is empty

    # Handle missing values (if any)
    df.replace({'orig_bytes': '-'}, '0', inplace=True)
    df['orig_bytes'] = pd.to_numeric(df['orig_bytes'], errors='coerce')
    df['orig_bytes'] = df['orig_bytes'].fillna(0).astype('int64')

    df.replace({'resp_bytes': '-'}, '0', inplace=True)
    df['resp_bytes'] = pd.to_numeric(df['resp_bytes'], errors='coerce')
    df['resp_bytes'] = df['resp_bytes'].fillna(0).astype('int64')


    df.replace({'duration': '-'}, '0', inplace=True)
    df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
    df['duration'] = df['duration'].fillna(0).astype('float64')

    df['service'] = df['service'].replace('-', np.nan)
    df['history'] = df['history'].replace('-', np.nan)

    # Convert categorical variables to numerical using Label Encoding
    # Encode protocol and service types
    label_encoders = {}
    for column in ['proto', 'service', 'conn_state', 'history', 'local_orig', 'local_resp']:
        if column in df.columns:
            le = LabelEncoder()
            df[column] = le.fit_transform(df[column])
            label_encoders[column] = le
    
    # Split df into features and labels
    X = df.drop(columns=['Attack'])  # Assuming 'label' is the target variable
    y = df['Attack']
    
    accuracy, f1, precision, recall =[], [], [], []
    skf= StratifiedKFold(n_splits=5,random_state=None)
    skf.get_n_splits(X,y)
    
    for (train_index, test_index), i in zip(skf.split(X, y), range(5)):
        X_train,X_test=X.iloc[train_index],X.iloc[test_index]
        y_train,y_test=y.iloc[train_index],y.iloc[test_index]


        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        
        
        # Map unseen values to '<unknown>'
        y_test = y_test.map(lambda s: '<unknown>' if s not in le.classes_ else s)
        # Add '<unknown>' to classes and transform
        le.classes_ = np.append(le.classes_, '<unknown>')
        y_test = le.transform(y_test)
        
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Initialize and train Extra Trees Classifier
        clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        
        f1Score = f1_score(y_true=y_test, y_pred=pred, average='macro')
        accScore=accuracy_score(y_test, pred)
        precScore = precision_score(y_test, pred, average='macro')
        recScrore = recall_score(y_test, pred, average='macro')
                             
        f1.append(f1Score)
        accuracy.append(accScore)
        precision.append(precScore)
        recall.append(recScrore)
        print('Fold: ', i, 'done!')

    meanScores, stdScores = {}, {}
    
    meanScores['f1Mean'] = np.array(f1).mean()
    meanScores['accMean'] = np.array(accuracy).mean()
    meanScores['recMean'] = np.array(recall).mean()
    meanScores['precMean'] = np.array(precision).mean()
    
    stdScores['f1Std'] = np.array(f1).std()
    stdScores['accStd'] = np.array(accuracy).std()
    stdScores['recStd'] = np.array(recall).std()
    stdScores['precStd'] = np.array(precision).std()
    
    print("Mean of all scores: ", meanScores)
    print("Std of all scores: ", stdScores)


    if save:
        save_scores(timeout, meanScores, stdScores)

    if meanScores['f1Mean'] > best_f1: 
        best_timeout = timeout
        best_mean = meanScores
        best_std = stdScores
        best_f1 = meanScores['f1Mean']
    
    if meanScores['f1Mean'] <= worst_f1: 
        
        worst_timeout = timeout
        worst_mean = meanScores
        worst_std = stdScores
        worst_f1 = meanScores['f1Mean']
               
    print('_______________________________________________')

Processing timeout :  default
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scores:  {'f1Mean': 0.755946861307675, 'accMean': 0.9626436590672371, 'recMean': 0.7904109296727396, 'precMean': 0.7488829655318966}
Std of all scores:  {'f1Std': 0.09661363149109703, 'accStd': 0.04081010407661601, 'recStd': 0.06338251556389947, 'precStd': 0.10465507795078655}
_______________________________________________
Processing timeout :  0.5
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scores:  {'f1Mean': 0.7079620561730767, 'accMean': 0.9643754252428364, 'recMean': 0.7591513396815561, 'precMean': 0.7040100450527869}
Std of all scores:  {'f1Std': 0.09686507529272413, 'accStd': 0.049830747081581105, 'recStd': 0.04628837776320015, 'precStd': 0.11157918472681805}
_______________________________________________
Processing timeout :  1
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all score

In [7]:
print("Best Timeout Combination: ", best_timeout)
print("Mean Scores (Best): ", best_mean)
print('Std Scores (Best):', best_std)

Best Timeout Combination:  5
Mean Scores (Best):  {'f1Mean': 0.7781845519616928, 'accMean': 0.9611703674470713, 'recMean': 0.8184170813025403, 'precMean': 0.7686564067381125}
Std Scores (Best): {'f1Std': 0.11090665308025613, 'accStd': 0.043323876785198966, 'recStd': 0.055640346447191534, 'precStd': 0.12420782381793535}


In [8]:
print("worst Timeout Combination: ", worst_timeout)
print("Mean Scores (Worst): ", worst_mean)
print('Std Scores (Worst):', worst_std)

worst Timeout Combination:  0.5
Mean Scores (Worst):  {'f1Mean': 0.7079620561730767, 'accMean': 0.9643754252428364, 'recMean': 0.7591513396815561, 'precMean': 0.7040100450527869}
Std Scores (Worst): {'f1Std': 0.09686507529272413, 'accStd': 0.049830747081581105, 'recStd': 0.04628837776320015, 'precStd': 0.11157918472681805}


In [9]:

results = {
    'Best score': {
        'Best Timeout': best_timeout,
        'Mean Scores (Best)': best_mean,
        'Std Scores (Best)': best_std,
    },
    
    'Worst score': {
        'Worst Timeout': worst_timeout,
        'Mean Scores (Worst)': worst_mean,
        'Std Scores (Worst)': worst_std,
    },
    
    'Difference': {
        'Accuracy': (best_mean['accMean'] - worst_mean['accMean'])*100,
        'F1 Score': (best_mean['f1Mean'] - worst_mean['f1Mean'])*100,
        'Precision': (best_mean['precMean'] - worst_mean['precMean'])*100,
        'Recall': (best_mean['recMean'] - worst_mean['recMean'])*100
    }
}



with open('../results/ET_cupid_zeek.json', 'w') as f:
    json.dump(results, f, indent=4)